In [ ]:
pip install pandas nltk textblob scikit-learn lime joblib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=a2a80585376188a227df9294276227033c2c90961d5e1606f8bb0c5317721620
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
# import libraries
import pandas as pd
import re
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import lime


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
df = pd.read_csv("dataset - modified - equal - v3.csv")
df.head()

,hate_speech_count,tweet
0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
1,1,""" @rhythmixx_ :hobbies include: fighting Maria..."
2,1,""" bitch get up off me """
3,1,""" bitch who do you love """
4,1,""" these hoes like niggas that spend money not ..."


In [ ]:
#Data Cleaning
def remove_unwanted_text(content):
    handle = re.sub('@[^\s]+', '', content)
    link = re.sub('http[^\s]+', '', handle)
    link = re.sub('www[^\s]+', '', link)
    ht = re.sub('#[^\s]+', '', link)
    final = re.sub('&[^\s]+', '', ht)
    return final

def remove_punctuations(words):
    new_words = []
    for w in words:
        l = re.sub('[^A-Za-z ]+', '', w)
        if l != '':
            new_words.append(l)
    return new_words

def remove_stop_words(words):
    sw = stopwords.words("english")
    sw = remove_punctuations(sw)
    return [w for w in words if w.lower() not in sw]

def stem_words(words):
    from nltk.stem import PorterStemmer
    from nltk.tokenize import word_tokenize
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def clean_data(content):
    cleaned_data = []
    for i in range(len(content)):

        tweet = remove_unwanted_text(content[i].lower())
        tb = TextBlob(tweet)
        words = remove_punctuations(tb.words)
        lemmatized_words = lemmatize_words(words)
        final_words = remove_stop_words(lemmatized_words)
        cleaned_data.append(" ".join(lemmatized_words))
    return cleaned_data



In [ ]:
df['clean_tweet'] = clean_data(df['tweet'])
df['clean_tweet'][9]

'ppl talk bad about the ghettohood but a a kid growing up a nigga had funnnnnnn'

In [ ]:
#Bag-of-Words
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['clean_tweet']).toarray()
y = df['hate_speech_count']

vocabulary_size = len(vectorizer.vocabulary_)

print("Number of features (unique words) before limiting:", vocabulary_size)

Number of features (unique words) before limiting: 11195


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression



In [ ]:
#Naive Bayes with BoW
#k-fold
for i in range(20):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
  naive_bayes_model = MultinomialNB()
  naive_bayes_model.fit(X_train, y_train)

  y_pred = naive_bayes_model.predict(X_test)
  def confusion_matrix(y_test, y_pred):
    cm = [[0, 0],
          [0, 0]]

    for true, pred in zip(y_test, y_pred):
        if true == 1 and pred == 1:
            cm[1][1] += 1
        elif true == 1 and pred == 0:
            cm[1][0] += 1
        elif true == 0 and pred == 1:
            cm[0][1] += 1
        elif true == 0 and pred == 0:
            cm[0][0] += 1

    return cm
  cm = confusion_matrix(y_test, y_pred)

  def calculate_metrics(confusion_matrix):
      TP = confusion_matrix[1][1]
      FP = confusion_matrix[0][1]
      TN = confusion_matrix[0][0]
      FN = confusion_matrix[1][0]

      precision = TP / (TP + FP) if (TP + FP) > 0 else 0
      recall = TP / (TP + FN) if (TP + FN) > 0 else 0
      f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
      return precision, recall, f1_score

  precision, recall, f1_score = calculate_metrics(cm)
  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy)
  print("Precision:", precision)
  print("Recall:", recall)
  print("F1-score:", f1_score)



Accuracy: 0.735820895522388
Precision: 0.6800966962127317
Recall: 0.8629856850715747
F1-score: 0.7607030193780983
Accuracy: 0.7572139303482587
Precision: 0.718801996672213
Recall: 0.8520710059171598
F1-score: 0.779783393501805
Accuracy: 0.7477611940298508
Precision: 0.7012048192771084
Recall: 0.8660714285714286
F1-score: 0.7749667110519308
Accuracy: 0.744776119402985
Precision: 0.695
Recall: 0.8501529051987767
F1-score: 0.7647867950481431
Accuracy: 0.7398009950248756
Precision: 0.6936936936936937
Recall: 0.8504016064257028
F1-score: 0.7640956247180877
Accuracy: 0.7577114427860696
Precision: 0.7139323990107173
Recall: 0.8608349900596421
F1-score: 0.7805317710680486
Accuracy: 0.7383084577114428
Precision: 0.6858974358974359
Recall: 0.8646464646464647
F1-score: 0.7649687220732797
Accuracy: 0.7368159203980099
Precision: 0.6919354838709677
Recall: 0.8537313432835821
F1-score: 0.7643652561247215
Accuracy: 0.7601990049751244
Precision: 0.7166392092257001
Recall: 0.8630952380952381
F1-score: 0

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF
X_train, X_test, y_train, y_test = train_test_split(df['clean_tweet'], df['hate_speech_count'], test_size=0.2, random_state=42)
tfidf_vectorizer = TfidfVectorizer()

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
#Naive Bayes with TF-IDF

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)


y_pred = nb_classifier.predict(X_test_tfidf)


In [ ]:

cm = confusion_matrix(y_test, y_pred)
precision, recall, f1_score = calculate_metrics(cm)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)

Accuracy: 0.7398009950248756
Precision: 0.7058340180772391
Recall: 0.8388671875
F1-score: 0.7666220437304775
